In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU is available:", gpus)
else:
    print("❌ GPU not detected")
    
Train_Dir = 'images/train'
Test_Dir = 'images/test'


TensorFlow version: 2.10.0
Keras version: 2.10.0
✅ GPU is available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(Train_Dir)

print(train)

test = pd.DataFrame()
test['image'], test['label'] = createdataframe(Test_Dir)

print(test)



angry completed
happy completed
neutral completed
sad completed
                              image  label
0          images/train\angry\0.jpg  angry
1          images/train\angry\1.jpg  angry
2         images/train\angry\10.jpg  angry
3      images/train\angry\10002.jpg  angry
4      images/train\angry\10016.jpg  angry
...                             ...    ...
21072     images/train\sad\9966.jpg    sad
21073     images/train\sad\9974.jpg    sad
21074     images/train\sad\9976.jpg    sad
21075     images/train\sad\9986.jpg    sad
21076     images/train\sad\9997.jpg    sad

[21077 rows x 2 columns]
angry completed
happy completed
neutral completed
sad completed
                            image  label
0     images/test\angry\10052.jpg  angry
1     images/test\angry\10065.jpg  angry
2     images/test\angry\10079.jpg  angry
3     images/test\angry\10095.jpg  angry
4     images/test\angry\10121.jpg  angry
...                           ...    ...
5135     images/test\sad\9864.jpg    sad
51

In [14]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, color_mode='grayscale', target_size=(48, 48))
        img_array = img_to_array(img)
        features.append(img_array)
    features = np.array(features, dtype='float32')
    return features

train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

x_train = train_features/255.0
x_test = test_features/255.0

le = LabelEncoder()
le.fit(train['label'])

y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test,num_classes=7)


100%|██████████| 5140/5140 [00:01<00:00, 2751.47it/s]


In [15]:
model = Sequential()
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam


In [16]:
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# --- Compile ---
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# --- Callbacks ---
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=15,              # Stop if no improvement in 15 epochs
        restore_best_weights=True # Revert to best model
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,               # Reduce LR by 0.3x
        patience=5,               # Wait 5 epochs before reducing LR
        min_lr=1e-6               # Minimum learning rate
    )
]

# --- Train ---
history = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=300,                   # can run safely; EarlyStopping will stop automatically
    validation_data=(x_test, y_test),
    callbacks=callbacks
)


Epoch 1/300
165/165 [==============================] - 8s 45ms/step - loss: 1.4380 - accuracy: 0.3061 - val_loss: 1.3670 - val_accuracy: 0.3551 - lr: 0.0010
Epoch 2/300
165/165 [==============================] - 6s 38ms/step - loss: 1.3652 - accuracy: 0.3408 - val_loss: 1.3274 - val_accuracy: 0.3652 - lr: 0.0010
Epoch 3/300
165/165 [==============================] - 6s 38ms/step - loss: 1.3203 - accuracy: 0.3816 - val_loss: 1.2553 - val_accuracy: 0.4292 - lr: 0.0010
Epoch 4/300
165/165 [==============================] - 6s 38ms/step - loss: 1.2213 - accuracy: 0.4520 - val_loss: 1.1113 - val_accuracy: 0.5134 - lr: 0.0010
Epoch 5/300
165/165 [==============================] - 6s 38ms/step - loss: 1.1550 - accuracy: 0.4918 - val_loss: 1.0592 - val_accuracy: 0.5424 - lr: 0.0010
Epoch 6/300
165/165 [==============================] - 6s 38ms/step - loss: 1.0952 - accuracy: 0.5229 - val_loss: 1.0119 - val_accuracy: 0.5673 - lr: 0.0010
Epoch 7/300
165/165 [==============================] - 6s 

In [17]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [18]:
from keras.models import model_from_json

In [19]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [20]:
label = ['angry','happy','neutral','sad']

In [21]:
def ef(image):
    img = load_img(image,grayscale=True)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [23]:
image = 'images/train/sad/3.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)

original image is of sad
1/1 [==============================] - 0s 66ms/step
model prediction is sad


c:\Users\Akshat\Desktop\PROJECT\Cat_Exp\.venv\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
